# Imputación de datos faltantes
En este cuaderno se aborda la imputacion con la tecnica de imputar la serie con valores dentro de la distribucion normal de cada mes, posteriormente entrenando el modelo SARIMA y finalmente imputando la serie original con el modelo entrenado.

In [1]:
import matplotlib.pyplot as plt
import shutil
import os
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [2]:
lista_zonas = ['MAGDALENA_MEDIO']
path = '../datos/separados_por_estacion/'

In [3]:
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf
from pmdarima import auto_arima

for zona in lista_zonas:
    print('Zona:',zona)
    path_zone = path+zona

    output_data_path = f'../datos/imputados_SARIMA/{zona}/'
    
    if os.path.exists(output_data_path):
        shutil.rmtree(output_data_path, ignore_errors=True)
    if not os.path.exists(output_data_path):
        os.makedirs(output_data_path)

    for file in os.listdir(path_zone):
        output_text = []
        print('Archivo:',file)
        station = file.split('.')[0]
        df = pd.read_csv(path_zone+'/'+file)

        df_ARIMA = df[["Fecha", "Mes", station]]
        df_ARIMA['Fecha'] = pd.to_datetime(df_ARIMA['Fecha'])

        df_ARIMA = df_ARIMA.rename(columns={station: 'Precipitación_original'})
        
        df_ARIMA['Precipitación_imputada'] = df_ARIMA["Precipitación_original"]
        months = df_ARIMA['Mes'].unique()
        for i in months:
            mean_month = df_ARIMA.loc[df_ARIMA['Mes'] == i]['Precipitación_imputada'].mean()
            std_month = df_ARIMA.loc[df_ARIMA['Mes'] == i]['Precipitación_imputada'].std()

            nulls_month = df_ARIMA[df_ARIMA['Mes']==i].isna().index

            vals_generated = np.random.normal(loc=mean_month, scale=std_month, size=len(nulls_month))
            for index, val in enumerate(nulls_month):
                df_ARIMA['Precipitación_imputada'].loc[val] = vals_generated[index]

        result = adfuller(df_ARIMA['Precipitación_imputada'])

        output_text.append(f'Estacion: {station}')
        output_text.append('ADF Statistic: %f' % result[0])
        output_text.append('p-value: %f' % result[1])
        output_text.append('Critical Values:')
        for key, value in result[4].items():
            output_text.append('\t%s: %.3f' % (key, value))
        output_text.append('\n')

        output_path = f'../salidas/{zona}/{station}/SARIMA/'
        if os.path.exists(output_path):
            shutil.rmtree(output_path, ignore_errors=True)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        # Guardar el texto en un archivo
        with open(f'../salidas/{zona}/{station}/SARIMA/ADF.txt', 'w') as f:
            for line in output_text:
                f.write(line)
                f.write("\n")

        # Graficar la PACF
        plt.figure(figsize=(10, 6))
        plot_pacf(df_ARIMA['Precipitación_imputada'], lags=40)
        plt.xlabel('Retrasos')
        plt.ylabel('Autocorrelación Parcial')
        plt.title('Autocorrelograma Parcial de la Precipitación')
        plt.savefig(f'{output_path}PACF.png')
        plt.close()

        model = auto_arima(df_ARIMA['Precipitación_imputada'], seasonal=True, m=12, trace=False)
        # Ajustamos el modelo ARIMA con los 6 valores encontrados
        model = ARIMA(df_ARIMA['Precipitación_imputada'], order=(model.order[0], model.order[1], model.order[2]), seasonal_order=(model.seasonal_order[0], model.seasonal_order[1], model.seasonal_order[2], 12))
        model_fit = model.fit()

        # Guardamos el resumen del modelo
        with open(f'{output_path}summary.txt', 'w') as f:
            f.write(str(model_fit.summary()))
        # Guardamos el modelo
        model_fit.save(f'{output_path}model.pkl')

        # Crear una copia del DataFrame original para no modificar los datos originales
        df_filled = df.copy()
        df_filled['Precipitación_original'] = df_filled[station]

        # Usar el modelo entrenado para predecir los valores nulos
        for i in df_filled[df_filled[station].isna()].index:
            df_filled.loc[i, station] = model_fit.predict(start=i, end=i).values[0]

        df_filled = df_filled.rename(columns={station: 'Precipitación_imputada'})

        df_filled['Precipitación_imputada'] = df_filled['Precipitación_imputada'].apply(lambda x: 0 if x < 0 else x)

        indices = np.arange(len(df_filled))
        coef = np.polyfit(indices, df_filled['Precipitación_imputada'], 1)  # coef = [pendiente, intersección]
        df_filled['xlin'] = np.polyval(coef, indices)

        # modificamos la fecha para mostrar solo yyyy-mm, casteamos antes a fecha
        df_filled['Fecha'] = pd.to_datetime(df_filled['Fecha']).apply(lambda x: x.strftime('%Y-%m'))

        # Graficamos la serie de tiempo original y la serie de tiempo imputada, imprimos con los indices en als etiquetas rotados 90 gradosç
        plt.figure(figsize=(10, 6))
        plt.plot(df_filled['Fecha'], df_filled['Precipitación_imputada'], label='Serie de Tiempo Imputada')
        plt.plot(df_filled['Fecha'], df_filled['Precipitación_original'], label='Serie de Tiempo Original')
        plt.plot(df_filled['Fecha'], df_filled['xlin'], label='Línea de Tendencia')
        plt.title('Serie de Tiempo Original vs Serie de Tiempo Imputada')
        plt.xlabel('Fecha (YYYY-MM)')
        plt.ylabel('Precipitación (mm)')
        plt.xticks(df_filled['Fecha'][::30])
        plt.legend()
        plt.savefig(f'{output_path}Serie_Tiempo_imputada.png')
        plt.close()

        df_filled['xint'] = model_fit.predict(start=0, end=len(df_filled)-1)
        df_filled['RESIDUALS'] = df_filled['Precipitación_original'] - df_filled['xint']
        df_filled.to_csv(f'{output_data_path}{station}.csv', index=False)


Zona: MAGDALENA_MEDIO
Archivo: 23010020.csv
Archivo: 23010080.csv
Archivo: 23020080.csv
Archivo: 23020090.csv
Archivo: 23020100.csv
Archivo: 23025040.csv
Archivo: 23040070.csv
Archivo: 23050080.csv
Archivo: 23050100.csv
Archivo: 23050230.csv
Archivo: 23050250.csv
Archivo: 23055040.csv
Archivo: 23055070.csv
Archivo: 23060110.csv
Archivo: 23060140.csv
Archivo: 23060150.csv
Archivo: 23060160.csv
Archivo: 23060170.csv
Archivo: 23060180.csv
Archivo: 23060190.csv
Archivo: 23060200.csv
Archivo: 23060260.csv
Archivo: 23060290.csv
Archivo: 23065120.csv
Archivo: 23070010.csv
Archivo: 23070020.csv
Archivo: 23080390.csv
Archivo: 23080640.csv
Archivo: 23080650.csv
Archivo: 23080720.csv
Archivo: 23080740.csv
Archivo: 23080750.csv
Archivo: 23080760.csv


c:\Users\nlmal\OneDrive - Universidad de La Rioja\Documentos\Master\Trabajo Fin de Master\HomogenizacionSeriesPrecipitacion\TFM\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Archivo: 23080810.csv
Archivo: 23080820.csv
Archivo: 23080920.csv
Archivo: 23080940.csv
Archivo: 23085030.csv
Archivo: 23085080.csv
Archivo: 23085110.csv
Archivo: 23085140.csv
Archivo: 23085160.csv
Archivo: 23085220.csv
Archivo: 23085270.csv
Archivo: 23090020.csv


c:\Users\nlmal\OneDrive - Universidad de La Rioja\Documentos\Master\Trabajo Fin de Master\HomogenizacionSeriesPrecipitacion\TFM\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Archivo: 23090110.csv
Archivo: 23100030.csv
Archivo: 23100040.csv
Archivo: 23100050.csv
Archivo: 23100070.csv
Archivo: 23105030.csv
Archivo: 23110030.csv
Archivo: 23110040.csv
Archivo: 23110060.csv
Archivo: 23120010.csv
Archivo: 23120050.csv
Archivo: 23120200.csv
Archivo: 23120210.csv
Archivo: 23120220.csv
Archivo: 23120240.csv


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>